## Outils pour la manipulation d'images et librairies.


In [273]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')



## Normalisation de l'image (YCbCr et padding)

Question 1 : Donner le code qui transforme une image RGB en une image YCbCr. Vous pourrez produire une matrice pour chaque composante, pour pouvoir plus facilement les manipuler indépendamment. Vous pouvez stocker les données YCbCr comme des entiers ou des flottants, mais vous expliquerez votre choix.

Réponse: 

Nous avons choisi de stocker les donées YCnCr comme des flottants car lors de la conversion de l'image RGB en YCbCr, on utilise des coefficients très précis. Or les conversions répétées entre les différents modèles de couleur pourrait entraîner une dégradation progressive de l'image.En stockant les données YCbCr en flottants, on peut éviter la perte de précision puisque le nombres flottants offrent une représentation plus précise des valeurs qui est nécessaire au vu de la précision des coefficients utilisés.

In [274]:
def RGB_en_YCbCR(image):
    image_finale = np.array(image, dtype='float64')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            R = image[i, j, 0]
            G = image[i, j, 1]
            B = image[i, j, 2]
            image_finale[i, j, 0] = 0.299 * R + 0.587 * G + 0.114 * B
            image_finale[i, j, 1] = -0.1687 * R - 0.3313 * G + 0.5 * B + 128
            image_finale[i, j, 2] = 0.5 * R - 0.4187 * G - 0.0813 * B + 128
    return image_finale


Question 2 : Donner le code qui transforme une image YCbCr en une image RGB. Attention, les valeurs des
canaux RGB doivent être un entier dans [0, 255] qui pourra être codé sur un octet. Appliquer successivement
la transformation RGB vers YCbCr puis YCbCr vers RGB et vérifier que vous obtenez l’image de départ.
Vous consulterez la documentation des fonctions de numpy clip, uint8 et mask qui pourraient vous être
utiles.

In [275]:
def YCbCR_en_RGB(image):
    image_finale = np.array(image, dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            Y = image[i, j, 0]
            Cb = image[i, j, 1]
            Cr = image[i, j, 2]
            image_finale[i, j, 0] = Y + 1.402 * (Cr - 128)
            image_finale[i, j, 1] = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
            image_finale[i, j, 2] = Y + 1.772 * (Cb - 128)
    return np.uint8(np.clip(image_finale, 0, 255))


Question 3 : Donner la fonction qui réalise ce padding ainsi que celle qui l’élimine et vérifier que
l’application de ces deux transformations laissent l’image inchangée.

In [276]:

def padding(image):
    padded_image = np.array(image, dtype='uint8')
    ligne = padded_image.shape[0]
    colonne = padded_image.shape[1]
    canaux = padded_image.shape[2]
    if ligne % 8 != 0:
        padded_image = np.concatenate((padded_image, np.zeros((ligne%8, colonne, canaux), dtype=np.uint8)), axis=0)
    if colonne % 8 != 0:
        padded_image = np.concatenate((padded_image, np.zeros((ligne, colonne%8, canaux), dtype=np.uint8)), axis=1)
    return padded_image

In [277]:
def unpadding(padded_image, image_origine):
    ligne = padded_image.shape[0]
    colonne = padded_image.shape[1]

    ligne_origine = image_origine.shape[0]
    colonne_origine = image_origine.shape[1]

    # Calculer le nombre de colonnes à enlever
    nb_col_enlever = colonne - colonne_origine
    while nb_col_enlever > 0:
        padded_image = padded_image[:, :-1]
        nb_col_enlever -= 1

    # Calculer le nombre de lignes à enlever
    nb_ligne_enlever = ligne- ligne_origine
    while nb_ligne_enlever > 0:
        padded_image = padded_image[:-1, :]
        nb_ligne_enlever -= 1

    return padded_image

Question 4 : Implémenter la fonction qui sous-échantillonne une matrice et renvoie une matrice deux fois
plus petite.

In [278]:
def sous_echantillonnage(matrice):
    matrice2 = matrice.copy()
    for i in range(matrice.shape[0]):
        for j in range(0, matrice.shape[1]):
            matrice[i, j, 1] = 0
            matrice[i, j, 2] = 0
    for i in range(matrice.shape[0]):
        for j in range(0, matrice.shape[1], 2):
            new_valueCb = (matrice2[i][j][1] + matrice2[i][j+1][1])//2
            new_valueCr = (matrice2[i][j][2] + matrice2[i][j+1][2])//2
            matrice[i, j//2, 1] = new_valueCb
            matrice[i, j//2, 2] = new_valueCr
    return matrice

Question 5 : Implémenter la fonction qui multiplie par deux la deuxième dimension d’une matrice. Tester à la suite le sous-échantillonnage et cette fonction, vous devez retrouver une image presque identique à celle de départ.

In [279]:

def mult_mat(matrice):
    matrice2 = matrice.copy()
    for i in range(matrice.shape[0]):
        for j in range(0, matrice.shape[1]//2):
            matrice[i][j*2][1] = matrice2[i][j][1]
            matrice[i][j*2][2] = matrice2[i][j][2]
            matrice[i][j*2+1][1] = matrice2[i][j][1]
            matrice[i][j*2+1][2] = matrice2[i][j][2]
    return matrice

## Découpage en blocs et compression

Question 6 : Soit une matrice dont les deux dimensions sont divisibles par 8. Donner une fonction qui découpe cette matrice en blocs 8 × 8 et les stocke dans une liste. L’ordre des blocs correspond à l’ordre de lecture d’une image.

In [280]:
def decoupe_bloc(matrice):
    lignes = matrice.shape[0]
    colonnes = matrice.shape[1]
    blocs = []
    blocs.clear()
    
    # Parcours des indices de début de chaque bloc
    for i in range(0, lignes, 8):
        for j in range(0, colonnes, 8):
            # Création d'un nouveau bloc vide
            bloc = []
            
            # Parcours des lignes du bloc
            for k in range(i, i + 8):
                # Extraction des éléments de la ligne correspondante
                ligne = matrice[k][j:j+8]
                # Ajout de la ligne au bloc
                bloc.append(ligne)
            
            # Ajout du bloc à la liste
            blocs.append(bloc)
    
    return blocs

Question 7 : Donner une fonction qui applique la transformée à chaque bloc d’une liste

In [281]:
def applique_trans(liste_bloc):
    liste_bloc = np.array(liste_bloc)
    for i in range(len(liste_bloc)):
        liste_bloc[i] = dct2(liste_bloc[i])
    return liste_bloc

Question 8 : Implémentez le filtrage des coefficients des blocs selon un seuil donné en argument.

In [282]:
def applique_seuil(liste_bloc, seuil):
    for i in range(len(liste_bloc)):
        for j in range(len(liste_bloc[0])):
            for k in range(len(liste_bloc[0][0])):
                for l in range(len(liste_bloc[0][0][0])):
                    if liste_bloc[i][j][k][l] < seuil:
                        liste_bloc[i][j][k][l] = 0
    return

Question 9 : Donner une fonction qui est capable, à partir d’une image RGB, de créer les listes de blocs compressés dans les 3 modes.

In [283]:
def trois_modes(image, seuil):
    mode_0 =applique_trans(decoupe_bloc(padding(RGB_en_YCbCR(image))))
    mode_1 = applique_seuil(applique_trans(decoupe_bloc(padding(RGB_en_YCbCR(image)))), seuil)
    mode_2 = applique_seuil(applique_trans(decoupe_bloc(padding(sous_echantillonnage(RGB_en_YCbCR(image))))), seuil)
    return mode_0, mode_1, mode_2

## Écriture dans un fichier

Question 10 : Pour commencer, vous  ́ecrirez quatres lignes contenant les informations de votre image. La première ligne contiendra le type du fichier : “SJPG”. La deuxième ligne contiendra les dimensions de l’image dans l’ordre hauteur puis largeur, séparées par un espace, par exemple “200 300”. La troisième ligne contiendra le mode de compression, par exemple “mode 1”. La quatrième ligne contiendra “RLE” si vous utilisez un run length encoding, ou “NORLE” sinon.

In [284]:
def ecrire_fichier(image, mode):
    f = open("fichier.txt", "w")
    f.write(f"SJPG\n{str(len(image[0]))} {str(len(image[0][0]))}\n{mode}\nRLE\n")

    f.close()
    return

Question 11 : Vous  ́ecrirez ensuite le contenu des blocs, d’abord ceux de Y, puis ceux de Cb puis ceux de Cr. Chaque bloc est écrit sur une ligne, les valeurs  ́etant des entiers séparés par de espaces.

In [285]:
def ecrire_bloc(image):
    f = open("fichier.txt", "a")
    for k in range(3):
        for i in range(len(image)):
            for j in range(len(image[0])):
                f.write(str(image[i][j][k])+" ")
        f.write('\n')
    f.close()
    return

test = load("test.png")
ecrire_bloc(test)

Question 12 : Ajouter une option à votre fonction d’écriture pour qu’elle puisse ́ecrire les blocs en appliquant le codage RLE.

In [286]:
def code_RLE(fichier):
    r = open(fichier, "r")
    e = open(fichier, "w")
    ligne = r.readlines()
    compt = 0
    for i in range(4,7):
        for car in ligne[i]:
            if car == '0':
                compt += 1
                ligne[i][car] = ""
            if car != '0':
                ligne[i][car] = "#"+compt
                compt=0
        e.writelines(ligne)
    r.close()
    e.close()
    return fichier

## Décompression

Question 13 :  ́Ecrire une fonction de décompression qui prend une liste de blocs pour chaque canal Y,Cb,Cr contenant les coefficients de la DCT et calcule une matrice représentant l’image en RGB.

In [287]:
def decompress(liste_bloc):
  num_blocks = len(liste_bloc)
  rows = int(np.sqrt(num_blocks))
  cols = num_blocks // rows
  image = np.zeros((rows*8, cols*8), dtype=np.uint8)

  for i in range(len(liste_bloc)):
    liste_bloc[i] = idct2(liste_bloc[i])

  for i in range(rows):
    for j in range(cols):
      block = liste_bloc[i*cols + j]
      image[i*8:(i+1)*8, j*8:(j+1)*8] = block

  return image

Question 14 :  ́Ecrire une fonction qui lit un fichier SJPG et qui créé les listes de blocs décrites par le fichier.

In [288]:
rle = False
def info_fichier(fichier):
    global rle
    f = open(fichier, "r")
    ligne = f.readlines()
    mode = ligne[2]
    if ligne[3] == "RLE":
        rle = True
    elif ligne[3] == "NORLE":
        rle = False
    f.close()
    return mode, rle

def decode_RLE(liste_bloc):
   for i in range(len(liste_bloc)):
       for j in range(len(liste_bloc[0])):
           for k in range(len(liste_bloc[0][0])):
               for l in range(len(liste_bloc[0][0][0])):
                   if liste_bloc[i][j][k][l] == "#":
                       liste_bloc[i][j][k][l] = 0
                   elif liste_bloc[i][j][k][l][0] == "#":
                       liste_bloc[i][j][k][l] = liste_bloc[i][j][k][l][1:]
   return liste_bloc


def lire_fichier(fichier):
    #creer les listes de blocs décrites par le fichier
    mode, rle = info_fichier(fichier)
    f = open(fichier, "r")
    ligne = f.readlines()
    liste_bloc = []
    liste_bloc.clear()
    for i in range(4,7):
        liste_bloc.append(ligne[i])
    if rle:
        liste_bloc = decode_RLE(liste_bloc)
    #replace /n by nothing
    for i in range(len(liste_bloc)):
        for j in range(len(liste_bloc[0])):
            for k in range(len(liste_bloc[0][0])):
                for l in range(len(liste_bloc[0][0][0])):
                    liste_bloc[i][j][k][l] = liste_bloc[i][j][k][l].replace("\n", "")
    liste_bloc = decompress(liste_bloc)
    f.close()
    return liste_bloc

## Tests 

In [289]:
test = load("test.png")

#testYCbCr = YCbCR_en_RGB(RGB_en_YCbCR(test))
#Image.fromarray(test,'RGB').show()
#Image.fromarray(testYCbCr,'RGB').show()

#padded_image = padding(testYCbCr)

#Image.fromarray(padded_image,'RGB').show()
#Image.fromarray(unpadding(padded_image, test),'RGB').show()

#Image.fromarray(YCbCR_en_RGB(mult_mat(sous_echantillonnage(RGB_en_YCbCR(test)))),'RGB').show()
#print(psnr(test, YCbCR_en_RGB(mult_mat(sous_echantillonnage(RGB_en_YCbCR(test))))))

mode_0 =applique_trans(decoupe_bloc(padding(RGB_en_YCbCR(test))))
mode_1 = applique_seuil(applique_trans(decoupe_bloc(padding(RGB_en_YCbCR(test)))),10)
mode_2 = applique_seuil(applique_trans(decoupe_bloc(padding(sous_echantillonnage(RGB_en_YCbCR(test))))),10)
ecrire_fichier(test, "mode 0")
ecrire_bloc(test)
info_fichier("fichier.txt")
lire_fichier("fichier.txt")
#Image.fromarray(mode_0,'RGB').show()
#Image.fromarray(mode_1,'RGB').show()

TypeError: 'str' object does not support item assignment